In [ ]:
https://adk-default-service-name-572950975329.us-central1.run.app

# list the agent

In [17]:
import requests
import json
# 2️⃣ GET request
url_get = "https://adk-default-service-name-572950975329.us-central1.run.app/list-apps"
response_get = requests.get(url_get)

print("GET response:", response_get.status_code, response_get.text)


GET response: 200 ["orchestrator"]


# check open sessions

In [6]:
import requests
from pprint import pprint

base = "https://adk-default-service-name-572950975329.us-central1.run.app"
url  = f"{base}/apps/orchestrator/users/steven/sessions"

r = requests.get(url)
r.raise_for_status()
payload = r.json()

def iter_sessions(payload):
    """Yield session dicts from various payload shapes."""
    if isinstance(payload, list):
        for s in payload:
            if isinstance(s, dict):
                yield s
    elif isinstance(payload, dict):
        # common containers
        for key in ("sessions", "items", "data", "results"):
            if key in payload and isinstance(payload[key], list):
                for s in payload[key]:
                    if isinstance(s, dict):
                        yield s
                return
        # map of id -> session
        for sid, s in payload.items():
            if isinstance(s, dict):
                s.setdefault("id", sid)
                yield s

def get_state_str(s):
    """Extract a state/status/phase as an UPPER string if present."""
    val = s.get("state") or s.get("status") or s.get("phase")
    if isinstance(val, str):
        return val.upper()
    if isinstance(val, dict):
        # try common fields within a nested object
        for k in ("name", "value", "label", "state", "status", "phase"):
            if isinstance(val.get(k), str):
                return val[k].upper()
    # boolean flags
    if s.get("open") is True or s.get("is_active") is True:
        return "ACTIVE"
    return ""

def get_end_time(s):
    """Unify possible end/close time fields."""
    return s.get("ended_at") or s.get("end_time") or s.get("closed_at")

sessions = list(iter_sessions(payload))

open_like = {"OPEN", "ACTIVE", "RUNNING"}
open_sessions = [s for s in sessions if (get_state_str(s) in open_like) or not get_end_time(s)]

print("All sessions:")
pprint(sessions)
print("========================")
print("\nOpen sessions:")
pprint(open_sessions)


All sessions:
[{'appName': 'orchestrator',
  'events': [],
  'id': 'sess-1',
  'lastUpdateTime': 1762579681.3664064,
  'state': {'init': 'ok'},
  'userId': 'steven'}]

Open sessions:
[{'appName': 'orchestrator',
  'events': [],
  'id': 'sess-1',
  'lastUpdateTime': 1762579681.3664064,
  'state': {'init': 'ok'},
  'userId': 'steven'}]


# IF NEEDED create open sesssion

In [5]:
import requests
import json

# 1️⃣ POST request (with JSON payload)
url_post = "https://adk-default-service-name-572950975329.us-central1.run.app/apps/orchestrator/users/steven/sessions/sess-1"
headers = {"Content-Type": "application/json"}
data = {"init": "ok"}

response_post = requests.post(url_post, headers=headers, json=data)
print("POST response:", response_post.status_code, response_post.text)


POST response: 200 {"id":"sess-1","appName":"orchestrator","userId":"steven","state":{"init":"ok"},"events":[],"lastUpdateTime":1762579681.3664064}


# talk to the agent

In [ ]:
# ---- imports (add if not already present) ----
from google.adk.runners import Runner
from google.adk.planners import BuiltInPlanner  # or PlanReActPlanner

# Reuse your existing services if you already create them elsewhere.
# For example, if you already have:
#   session_service = InMemorySessionService()  or your DB-backed service
#   artifact_service = InMemoryArtifactService()
# then point the Runner to them.
runner = Runner(
    session_service=session_service,         # <--- use your existing one
    artifact_service=artifact_service,       # <--- optional, use your existing one
    planner=BuiltInPlanner(),                # or PlanReActPlanner()
)

# ---- in your existing PATCH route: /apps/{appName}/users/{userId}/sessions/{sessionId} ----
@app.patch("/apps/{appName}/users/{userId}/sessions/{sessionId}")
def patch_session(appName: str, userId: str, sessionId: str, body: dict):
    # 1) Your existing code that applies the stateDelta to the session
    #    (this is already working for you)
    session = apply_state_delta(appName, userId, sessionId, body["stateDelta"])

    # 2) *** RUN THE AGENT NOW ***
    runner.run_until_idle(
        app_name=appName,
        user_id=userId,
        session_id=sessionId,
    )

    # 3) Return the updated session (which now should include assistant events)
    return get_session(appName, userId, sessionId)
